# A/B Testing with NFL Data: A Statistical Deep Dive

Welcome to this advanced session on A/B Testing, where we'll apply fundamental statistical concepts to NFL (National Football League) data. A/B testing is a powerful method for comparing two versions of something (A and B) to determine which one performs better. In the world of NFL, this could mean comparing two different training methods, two offensive plays, or even two marketing strategies for merchandise.

This notebook will guide you through key statistical topics essential for understanding and conducting A/B tests, including distributions, central measures, deviations, and the Central Limit Theorem. We'll then apply various hypothesis tests using `scipy.stats` to synthetic NFL-themed scenarios.

Let's start by importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests # For post-hoc correction

# Set display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print("Pandas, NumPy, Matplotlib, Seaborn, and SciPy Stats imported. Display options set.")

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10


## 1. Understanding Key Statistical Concepts

Before diving into A/B testing, let's recap some fundamental statistical concepts that form its backbone.

### 1.1. Distributions: Normal, t, and Chi-Square

Understanding the shape of your data is crucial. Different statistical tests assume different underlying distributions. Here, we'll look at three common ones:

* **Normal Distribution (Gaussian Distribution):** Often called the 'bell curve', it's symmetric around its mean, with data points closer to the mean being more frequent. Many natural phenomena follow this distribution.
* **t-Distribution (Student's t-distribution):** Similar to the normal distribution but with fatter tails, meaning it has more probability in the tails. It's used when the sample size is small and the population standard deviation is unknown. As the sample size increases, the t-distribution approaches the normal distribution.
* **Chi-Square ($\chi^2$) Distribution:** A non-symmetric distribution used primarily in hypothesis tests involving categorical data, like goodness-of-fit tests or tests of independence (which we'll use for the Chi-squared test).


#### Task 1: Visualize a Normal Distribution and a t-Distribution.

Plot a standard normal distribution and a t-distribution with a small degree of freedom (e.g., 5) together on the same plot to observe their differences.

In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Use `np.linspace` to create a range of x-values. For the normal distribution, use `stats.norm.pdf(x, loc=mean, scale=std_dev)`. For the t-distribution, use `stats.t.pdf(x, df=degrees_of_freedom)`. Plot both on the same `plt.figure`.
</details>

##### Solution

In [ ]:
x = np.linspace(-4, 4, 500)

# Normal Distribution (mean=0, std=1)
normal_pdf = stats.norm.pdf(x, loc=0, scale=1)

# t-Distribution (df=5)
t_pdf = stats.t.pdf(x, df=5)

plt.figure(figsize=(10, 6))
plt.plot(x, normal_pdf, label='Normal Distribution ($\mu=0, \sigma=1$)', color='blue')
plt.plot(x, t_pdf, label='t-Distribution (df=5)', color='red', linestyle='--')
plt.title('Normal vs. t-Distribution')
plt.xlabel('X')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

print("The plots show the characteristic shapes of the Normal and t-distributions. The t-distribution with fewer degrees of freedom has fatter tails and a lower peak compared to the normal distribution.")

#### Task 2: Visualize multiple Chi-Square Distributions.

Plot Chi-Square distributions with various degrees of freedom (e.g., 1, 2, 3, 5, 10) on the same plot to see how its shape changes with increasing degrees of freedom. Use the `scipy.stats.chi2.pdf` function.

In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Use `np.linspace` for x-values. Loop through a list of degrees of freedom and for each, calculate `stats.chi2.pdf(x, df=degrees_of_freedom)` and plot it on the same figure.
</details>

##### Solution

In [ ]:
x_chi2 = np.linspace(0, 20, 500)
dfs = [1, 2, 3, 5, 10]

plt.figure(figsize=(10, 6))
for df in dfs:
    chi2_pdf = stats.chi2.pdf(x_chi2, df=df)
    plt.plot(x_chi2, chi2_pdf, label=f'Chi-Square (df={df})')

plt.title('Chi-Square Distributions for Various Degrees of Freedom')
plt.xlabel('X')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

print("The Chi-Square distribution changes shape with degrees of freedom, becoming less skewed and more bell-shaped as df increases.")

### 1.2. Central Measures and Measures of Deviation

* **Central Measures:** Describe the central tendency of the data.
    * **Mean:** The average value. Sum of all values divided by the count of values.
    * **Median:** The middle value when data is sorted. Less affected by outliers.
    * **Mode:** The most frequently occurring value.
* **Measures of Deviation (Spread):** Describe how spread out the data is.
    * **Variance:** The average of the squared differences from the mean.
    * **Standard Deviation (sigma):** The square root of the variance. Indicates the typical distance of data points from the mean. Easier to interpret than variance as it's in the same units as the data.
    * **Standard Error of the Mean (SEM):** The standard deviation of the sampling distribution of the mean. It estimates how much the sample mean is likely to vary from the population mean. It's calculated as (sigma / sqrt(n)), where sigma is the population standard deviation and n is the sample size. If sigma is unknown, the sample standard deviation s is used: (s / sqrt(n)).


#### Task 3: Calculate central measures and measures of deviation for the `new_regimen_40_yd_dash` data.

Calculate the mean, median, mode, variance, standard deviation, and standard error of the mean for the `new_regimen_40_yd_dash` data.

##### Dataset: New Regimen 40-Yard Dash Times

This dataset represents the 40-yard dash times for a sample of 30 NFL players who have undergone a new, experimental training regimen. The goal is to evaluate if this new regimen leads to faster sprint times compared to the historical league average. We will use this data to calculate various central measures and measures of deviation to understand the performance of players under this new regimen.

In [ ]:
np.random.seed(42)
# Assume league average 40-yard dash time is 4.6 seconds with a std dev of 0.2
league_avg_40_yd_dash = 4.6
league_std_40_yd_dash = 0.2

# A new training regimen is tested on a sample of 30 players
# We'll simulate a slight improvement for the 'new regimen' group
new_regimen_40_yd_dash = pd.Series(np.random.normal(loc=4.5, scale=0.18, size=30))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Use Pandas Series methods like `.mean()`, `.median()`, `.mode()`, `.var()`, `.std()`, and `.sem()`.
</details>

##### Solution

In [ ]:
mean_dash = new_regimen_40_yd_dash.mean()
median_dash = new_regimen_40_yd_dash.median()
mode_dash = new_regimen_40_yd_dash.mode()[0] # .mode() returns a Series, take the first if multiple
variance_dash = new_regimen_40_yd_dash.var(ddof=1)
std_dev_dash = new_regimen_40_yd_dash.std(ddof=1)
sem_dash = new_regimen_40_yd_dash.sem()

print(f"Mean 40-yard dash time: {mean_dash:.3f} seconds")
print(f"Median 40-yard dash time: {median_dash:.3f} seconds")
print(f"Mode 40-yard dash time: {mode_dash:.3f} seconds")
print(f"Variance of 40-yard dash times: {variance_dash:.4f}")
print(f"Standard Deviation of 40-yard dash times: {std_dev_dash:.3f} seconds")
print(f"Standard Error of the Mean (SEM) for 40-yard dash times: {sem_dash:.3f} seconds")

#### Task 4: Explore another dataset: Player Touchdowns.

Generate synthetic data for player touchdowns and calculate their central measures and measures of deviation.

##### Dataset: Player Touchdowns

This dataset simulates the number of touchdowns scored by 100 different players over a season. It's designed to represent a slightly skewed distribution, common in sports statistics where a few players might score many touchdowns while most score fewer. We will analyze its central tendency and spread.

In [ ]:
np.random.seed(46)
player_touchdowns = pd.Series(np.random.poisson(lam=5, size=100) + np.random.randint(0, 3, size=100)) # Simulating a skewed distribution


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Apply Pandas Series methods like `.mean()`, `.median()`, `.mode()`, `.var()`, `.std()`, and `.sem()`.
</details>

##### Solution

In [ ]:
mean_tds = player_touchdowns.mean()
median_tds = player_touchdowns.median()
mode_tds = player_touchdowns.mode()[0]
variance_tds = player_touchdowns.var(ddof=1)
std_dev_tds = player_touchdowns.std(ddof=1)
sem_tds = player_touchdowns.sem()

print(f"Mean Touchdowns: {mean_tds:.2f}")
print(f"Median Touchdowns: {median_tds:.2f}")
print(f"Mode Touchdowns: {mode_tds:.2f}")
print(f"Variance of Touchdowns: {variance_tds:.2f}")
print(f"Standard Deviation of Touchdowns: {std_dev_tds:.2f}")
print(f"Standard Error of the Mean (SEM) for Touchdowns: {sem_tds:.2f}")

### 1.3. Central Limit Theorem (CLT)

The Central Limit Theorem is a cornerstone of statistics. It states that, given a sufficiently large sample size from a population with a finite level of variance, the mean of all samples from the same population will be approximately equal to the mean of the population. Furthermore, the distribution of these sample means will be approximately normally distributed, regardless of the shape of the original population distribution.

This is incredibly powerful because it allows us to use normal distribution theory for hypothesis testing even when the original population data isn't normally distributed, as long as our sample sizes are large enough (typically n >= 30).


#### Task 5: Demonstrate the Central Limit Theorem.

Take many samples from the `population_player_weights` (which is non-normally distributed) and plot the distribution of their means to observe the CLT in action. For this task, use a sample size of 100 and take 10,000 samples.

##### Dataset: Population Player Weights

This dataset represents the weights of a large population of NFL players. It's deliberately simulated as a bimodal distribution (e.g., lighter skill players and heavier linemen) to vividly demonstrate the Central Limit Theorem. We'll draw samples from this population to show how sample means tend towards a normal distribution, regardless of the original population's shape.

In [ ]:
np.random.seed(45)
# Simulate player weights (non-normal distribution, e.g., uniform or bimodal)
# We'll use a bimodal distribution to make the CLT effect more apparent
weights_group1 = np.random.normal(loc=200, scale=15, size=500) # Smaller players
weights_group2 = np.random.normal(loc=280, scale=20, size=500) # Larger players (OL/DL)
population_player_weights = np.concatenate((weights_group1, weights_group2))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Generate sample means for different combinations of sample sizes and number of samples. Plot their histograms in separate plots, focusing only on the distribution of sample means.
</details>

##### Solution

In [ ]:
sample_size_clt = 100
num_samples_clt = 10000
sample_means = []

for _ in range(num_samples_clt):
    sample = np.random.choice(population_player_weights, size=sample_size_clt, replace=False)
    sample_means.append(np.mean(sample))

plt.figure(figsize=(10, 6))
plt.hist(population_player_weights, bins=30, alpha=0.8, color='lightcoral')
plt.title('Population Distribution (Player Weights)')
plt.xlabel('Player Weight (lbs)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(sample_means, bins=30, alpha=0.8, color='skyblue')
plt.title(f'Distribution of Sample Means (n={sample_size_clt}, {num_samples_clt} samples)')
plt.xlabel('Sample Mean Weight (lbs)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

print("Observe how the distribution of sample means, even from a non-normal population, approximates a normal distribution.")
print(f"Population Mean: {np.mean(population_player_weights):.2f}")
print(f"Mean of Sample Means: {np.mean(sample_means):.2f}")
print(f"Standard Deviation of Sample Means (SEM): {np.std(sample_means, ddof=1):.2f}")
print(f"Theoretical SEM: {np.std(population_player_weights, ddof=1) / np.sqrt(sample_size_clt):.2f}")

#### Task 6: Explore the effect of sample size and number of samples on the CLT.

Repeat the CLT demonstration with a smaller sample size (e.g., 5) and a larger sample size (e.g., 100), and vary the number of samples (e.g., 1000 vs 10000) to visually compare the approximation to a normal distribution. Plot only the sample mean distributions in separate plots.

In [ ]:
np.random.seed(45)
# Simulate player weights (non-normal distribution, e.g., uniform or bimodal)
# We'll use a bimodal distribution to make the CLT effect more apparent
weights_group1 = np.random.normal(loc=200, scale=15, size=500) # Smaller players
weights_group2 = np.random.normal(loc=280, scale=20, size=500) # Larger players (OL/DL)
population_player_weights = np.concatenate((weights_group1, weights_group2))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

Generate sample means for different combinations of sample sizes and number of samples. Plot their histograms in separate plots, focusing only on the distribution of sample means.
</details>

##### Solution

In [ ]:
sample_size_options = [5, 100]
num_samples_options = [1000, 10000]

for s_size in sample_size_options:
    for n_samples in num_samples_options:
        sample_means_compare = []
        for _ in range(n_samples):
            sample = np.random.choice(population_player_weights, size=s_size, replace=False)
            sample_means_compare.append(np.mean(sample))

        plt.figure(figsize=(10, 6))
        plt.hist(sample_means_compare, bins=30, alpha=0.8, color='green' if s_size == 100 else 'orange')
        plt.title(f'Distribution of Sample Means (n={s_size}, {n_samples} samples)')
        plt.xlabel('Sample Mean Weight (lbs)')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.show()

print("These plots demonstrate that as the sample size increases, the distribution of sample means becomes more tightly clustered around the population mean and more closely approximates a normal distribution. Increasing the number of samples further refines this approximation.")

## 2. Hypothesis Testing Fundamentals for A/B Testing

A/B testing is essentially a form of hypothesis testing. We formulate hypotheses about the effect of a change and then use statistical tests to determine if there's enough evidence to support our claim.

### 2.1. Null and Alternative Hypotheses

* **Null Hypothesis (H0):** This is the default or status quo assumption. It states that there is no significant difference or no effect. In A/B testing, it often means that version A and version B are the same.
    * Example: "The new training regimen has no effect on 40-yard dash times; the average time for players on the new regimen is the same as the league average."
* **Alternative Hypothesis (H1 or HA):** This is what we are trying to prove. It states that there *is* a significant difference or an effect.
    * Example: "The new training regimen *does* affect 40-yard dash times; the average time for players on the new regimen is *different* from the league average." (Two-sided test)
    * Example: "The new training regimen *improves* 40-yard dash times; the average time for players on the new regimen is *less* than the league average." (One-sided test)


### 2.2. Confidence Levels and p-values

* **Confidence Level (alpha):** Also known as the significance level. This is the probability of rejecting the null hypothesis when it is actually true (Type I error). Common values are 0.05 (95% confidence) or 0.01 (99% confidence). If p <= alpha, we reject H0.
* **p-value:** The probability of observing a test statistic as extreme as, or more extreme than, the one calculated from the sample data, *assuming the null hypothesis is true*. A small p-value (typically p < alpha) suggests that the observed data is unlikely under the null hypothesis, leading us to reject H0. A large p-value suggests the observed data is consistent with H0, so we fail to reject H0.


In [ ]:
# Illustration of p-value
x_p_value = np.linspace(-3, 3, 500)
normal_dist_p_value = stats.norm.pdf(x_p_value, loc=0, scale=1)

plt.figure(figsize=(10, 6))
plt.plot(x_p_value, normal_dist_p_value, color='blue', label='Standard Normal Distribution')

# Simulate a test statistic and p-value
test_statistic_example = 1.8
p_value_example = 1 - stats.norm.cdf(test_statistic_example) # One-tailed p-value

# Shade the p-value area
x_shade = np.linspace(test_statistic_example, 3, 100)
plt.fill_between(x_shade, 0, stats.norm.pdf(x_shade, loc=0, scale=1), color='red', alpha=0.5, label=f'P-value area (p={p_value_example:.3f})')

plt.axvline(x=test_statistic_example, color='green', linestyle='--', label=f'Test Statistic = {test_statistic_example:.2f}')
plt.title('Illustration of P-value (One-tailed test)')
plt.xlabel('Z-score')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

print("This plot illustrates a p-value. The shaded red area represents the probability of observing a test statistic as extreme or more extreme than the one calculated, assuming the null hypothesis is true.")

### 2.2.1. Alpha, p-values, Critical Values, and Test Statistics

When performing a hypothesis test, we use two main approaches to decide whether to reject the null hypothesis:

1.  **Comparing p-value to Alpha (alpha):**
    * **Alpha (alpha)**: This is your predetermined significance level, often 0.05. It represents the maximum probability of making a Type I error (falsely rejecting the null hypothesis) that you are willing to accept.
    * **p-value**: This is the probability of observing your sample data (or more extreme data) if the null hypothesis were true. It's calculated from your sample data.
    * **Decision Rule**: If p <= alpha, you reject the null hypothesis. If p > alpha, you fail to reject the null hypothesis.

2.  **Comparing Test Statistic to Critical Value(s):**
    * **Test Statistic (e.g., t-score, z-score, chi2-score)**: This is a standardized value calculated from your sample data that measures how far your sample result deviates from what you would expect if the null hypothesis were true. For example:
        * **t-score**: Used in t-tests when the population standard deviation is unknown or sample size is small.
        * **z-score**: Used in z-tests when the population standard deviation is known or sample size is large (and thus the sample mean distribution approximates normal by CLT).
        * **chi2-score**: Used in Chi-squared tests for categorical data.
    * **Critical Value(s)**: These are one or two values (depending on whether it's a one-sided or two-sided test) that define the "rejection region" in the sampling distribution. They are determined by your chosen alpha level and the degrees of freedom (if applicable) of the test distribution.
    * **Decision Rule**: If your test statistic falls into the rejection region (i.e., is more extreme than the critical value(s)), you reject the null hypothesis. Otherwise, you fail to reject it.

Both methods lead to the same conclusion, as the p-value is derived from the test statistic's position relative to the distribution under the null hypothesis.

In [ ]:
# Illustration of Alpha, Critical Values, and Test Statistic (Two-sided Z-test)
x_alpha = np.linspace(-3, 3, 500)
normal_dist_alpha = stats.norm.pdf(x_alpha, loc=0, scale=1)

alpha_level = 0.05
critical_value_lower = stats.norm.ppf(alpha_level / 2)
critical_value_upper = stats.norm.ppf(1 - alpha_level / 2)

plt.figure(figsize=(10, 6))
plt.plot(x_alpha, normal_dist_alpha, color='blue', label='Standard Normal Distribution (H0)')

# Shade rejection regions
x_reject_lower = np.linspace(-3, critical_value_lower, 100)
plt.fill_between(x_reject_lower, 0, stats.norm.pdf(x_reject_lower, loc=0, scale=1), color='red', alpha=0.5, label=f'Rejection Region (α/2={alpha_level/2:.2f})')

x_reject_upper = np.linspace(critical_value_upper, 3, 100)
plt.fill_between(x_reject_upper, 0, stats.norm.pdf(x_reject_upper, loc=0, scale=1), color='red', alpha=0.5)

plt.axvline(x=critical_value_lower, color='red', linestyle=':', label=f'Critical Value = {critical_value_lower:.2f}')
plt.axvline(x=critical_value_upper, color='red', linestyle=':')

# Example Test Statistic (falls outside rejection region - fail to reject H0)
test_statistic_fail_reject = 1.0
plt.axvline(x=test_statistic_fail_reject, color='green', linestyle='--', label=f'Test Statistic = {test_statistic_fail_reject:.2f} (Fail to Reject H0)')

plt.title('Alpha, Critical Values, and Test Statistic (Two-sided Test)')
plt.xlabel('Z-score')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

# Example Test Statistic (falls inside rejection region - reject H0)
test_statistic_reject = 2.5
plt.figure(figsize=(10, 6))
plt.plot(x_alpha, normal_dist_alpha, color='blue', label='Standard Normal Distribution (H0)')
plt.fill_between(x_reject_lower, 0, stats.norm.pdf(x_reject_lower, loc=0, scale=1), color='red', alpha=0.5, label=f'Rejection Region (α/2={alpha_level/2:.2f})')
plt.fill_between(x_reject_upper, 0, stats.norm.pdf(x_reject_upper, loc=0, scale=1), color='red', alpha=0.5)
plt.axvline(x=critical_value_lower, color='red', linestyle=':')
plt.axvline(x=critical_value_upper, color='red', linestyle=':', label=f'Critical Value = {critical_value_upper:.2f}')
plt.axvline(x=test_statistic_reject, color='purple', linestyle='--', label=f'Test Statistic = {test_statistic_reject:.2f} (Reject H0)')

plt.title('Alpha, Critical Values, and Test Statistic (Two-sided Test)')
plt.xlabel('Z-score')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

print("These plots illustrate how the critical values define the rejection regions based on the alpha level. If the calculated test statistic falls within these shaded regions, we reject the null hypothesis.")

### 2.3. Type I and Type II Errors

When conducting hypothesis tests, there are two types of errors we can make:

* **Type I Error (alpha):** Rejecting the null hypothesis when it is actually true (False Positive). Also known as a "false alarm." In our NFL example, concluding the new training regimen *improves* speed when it actually doesn't.
* **Type II Error (beta):** Failing to reject the null hypothesis when it is actually false (False Negative). Also known as a "miss." In our NFL example, concluding the new training regimen *doesn't improve* speed when it actually does.

There's a trade-off between Type I and Type II errors. Decreasing the probability of one typically increases the probability of the other. The power of a test (1 - beta) is the probability of correctly rejecting a false null hypothesis.

### 2.4. Confidence Intervals

A **confidence interval** provides a range of values, derived from sample data, that is likely to contain the true value of an unknown population parameter. The confidence level (e.g., 95%) indicates the long-run frequency with which confidence intervals, constructed in this manner, contain the true parameter.

For example, a 95% confidence interval for a mean means that if we were to take many samples and compute a confidence interval for each, approximately 95% of those intervals would contain the true population mean.

In [ ]:
# Illustration of Confidence Interval
sample_mean_ci = 100
sample_std_ci = 10
n_ci = 30
sem_ci = sample_std_ci / np.sqrt(n_ci)
confidence_level_ci = 0.95
degrees_freedom_ci = n_ci - 1

conf_int_lower, conf_int_upper = stats.t.interval(
    confidence=confidence_level_ci,
    df=degrees_freedom_ci,
    loc=sample_mean_ci,
    scale=sem_ci
)

x_ci = np.linspace(sample_mean_ci - 3 * sem_ci, sample_mean_ci + 3 * sem_ci, 500)
t_dist_ci = stats.t.pdf(x_ci, df=degrees_freedom_ci, loc=sample_mean_ci, scale=sem_ci)

plt.figure(figsize=(10, 6))
plt.plot(x_ci, t_dist_ci, color='blue', label='Sampling Distribution of the Mean')

# Shade the confidence interval
x_fill_ci = np.linspace(conf_int_lower, conf_int_upper, 100)
plt.fill_between(x_fill_ci, 0, stats.t.pdf(x_fill_ci, df=degrees_freedom_ci, loc=sample_mean_ci, scale=sem_ci), color='skyblue', alpha=0.6, label=f'{confidence_level_ci*100:.0f}% Confidence Interval')

plt.axvline(x=sample_mean_ci, color='green', linestyle='--', label=f'Sample Mean = {sample_mean_ci:.2f}')
plt.axvline(x=conf_int_lower, color='red', linestyle=':', label=f'Lower Bound = {conf_int_lower:.2f}')
plt.axvline(x=conf_int_upper, color='red', linestyle=':', label=f'Upper Bound = {conf_int_upper:.2f}')

plt.title('Illustration of a Confidence Interval')
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()

print("This plot shows a confidence interval around a sample mean. The interval represents a range within which the true population mean is likely to fall with a certain level of confidence.")

#### Task 7: Calculate a 95% confidence interval for the mean of `new_regimen_40_yd_dash`.

Use `scipy.stats.t.interval` to calculate the confidence interval, as we are dealing with a sample and unknown population standard deviation.

In [ ]:
np.random.seed(42)
# Assume league average 40-yard dash time is 4.6 seconds with a std dev of 0.2
league_avg_40_yd_dash = 4.6
league_std_40_yd_dash = 0.2

# A new training regimen is tested on a sample of 30 players
# We'll simulate a slight improvement for the 'new regimen' group
new_regimen_40_yd_dash = pd.Series(np.random.normal(loc=4.5, scale=0.18, size=30))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The `stats.t.interval` function takes `confidence`, `df` (degrees of freedom, which is `n-1`), `loc` (sample mean), and `scale` (standard error of the mean) as arguments.
</details>

##### Solution

In [ ]:
confidence_level = 0.95
degrees_freedom = len(new_regimen_40_yd_dash) - 1
sample_mean = new_regimen_40_yd_dash.mean()
sample_std_error = new_regimen_40_yd_dash.sem()

confidence_interval = stats.t.interval(
    confidence=confidence_level,
    df=degrees_freedom,
    loc=sample_mean,
    scale=sample_std_error
)

print(f"Sample Mean: {sample_mean:.3f}")
print(f"95% Confidence Interval for 40-yard dash time: ({confidence_interval[0]:.3f}, {confidence_interval[1]:.3f})")
print("Interpretation: We are 95% confident that the true average 40-yard dash time for players on the new regimen falls within this interval.")

## 3. Applying Hypothesis Tests to NFL Data

Now, let's put these concepts into practice using our synthetic NFL data. We'll perform one-sided and two-sided tests where appropriate.

### 3.1. One-Sample t-Test (`scipy.stats.ttest_1samp`)

The one-sample t-test compares the mean of a single sample to a known population mean (or a hypothesized value). It's used when the population standard deviation is unknown.


#### Task 8: Perform a 1-sample t-test.

**Scenario 1:** A new training regimen is implemented to improve player speed. We want to know if the average 40-yard dash time of players on this new regimen is significantly *less* than the historical league average of 4.6 seconds.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Use `new_regimen_40_yd_dash` and `league_avg_40_yd_dash` to perform the test. Set alpha = 0.05. Use the `alternative` parameter to get the correct p-value directly.

##### Dataset: New Regimen 40-Yard Dash Times

This dataset represents the 40-yard dash times for a sample of 30 NFL players who have undergone a new, experimental training regimen. The goal is to evaluate if this new regimen leads to faster sprint times compared to the historical league average. We will use this data to calculate various central measures and measures of deviation to understand the performance of players under this new regimen.

In [ ]:
np.random.seed(42)
# Assume league average 40-yard dash time is 4.6 seconds with a std dev of 0.2
league_avg_40_yd_dash = 4.6
league_std_40_yd_dash = 0.2

# A new training regimen is tested on a sample of 30 players
# We'll simulate a slight improvement for the 'new regimen' group
new_regimen_40_yd_dash = pd.Series(np.random.normal(loc=4.5, scale=0.18, size=30))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that the mean 40-yard dash time for players on the new regimen is greater than or equal to 4.6 seconds. This implies a one-sided (left-tailed) test.

Use `stats.ttest_1samp(a=sample_data, popmean=population_mean, alternative='less')`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): The mean 40-yard dash time for players on the new regimen is equal to or greater than 4.6 seconds. (mu >= 4.6)")
print("Alternative Hypothesis (H1): The mean 40-yard dash time for players on the new regimen is less than 4.6 seconds. (mu < 4.6)\n")

t_statistic, p_value_one_sided = stats.ttest_1samp(a=new_regimen_40_yd_dash, popmean=league_avg_40_yd_dash, alternative='less')

print(f"Sample Mean 40-yard Dash: {new_regimen_40_yd_dash.mean():.3f}")
print(f"League Average 40-yard Dash (Population Mean): {league_avg_40_yd_dash:.1f}")
print(f"T-statistic: {t_statistic:.3f}")
print(f"One-sided (Left-tailed) P-value: {p_value_one_sided:.10f}")

print()
if p_value_one_sided < alpha:
    print(f"Since the one-sided p-value ({p_value_one_sided:.10f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is significant evidence that the new training regimen results in 40-yard dash times that are significantly LESS than the league average.")
else:
    print()
    print(f"Since the one-sided p-value ({p_value_one_sided:.10f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is not enough significant evidence to conclude that the new training regimen results in 40-yard dash times that are significantly LESS than the league average.")

#### Task 9: Perform a 1-sample t-test.

**Scenario 2:** A new football helmet design is being tested for its ability to reduce impact forces. The average impact force for standard helmets is known to be 100 Gs. We test a sample of new helmets and want to know if the average impact force is significantly *different* from 100 Gs.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Generate synthetic data for new helmet impact forces and perform the test. Set alpha = 0.05. Use the `alternative` parameter to get the correct p-value directly.

##### Dataset: New Helmet Impact Forces

This dataset contains simulated impact force measurements (in Gs) for a sample of 25 newly designed football helmets. The standard average impact force for traditional helmets is known to be 100 Gs. We aim to determine if the new helmet design significantly alters this average impact force, either increasing or decreasing it.

In [ ]:
np.random.seed(47)
new_helmet_impact_forces = pd.Series(np.random.normal(loc=98, scale=5, size=25))
known_avg_impact_force = 100


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that the mean impact force for new helmets is equal to 100 Gs. This implies a two-sided test.

Use `stats.ttest_1samp(a=sample_data, popmean=population_mean, alternative='two-sided')`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): The mean impact force for new helmets is equal to 100 Gs. (mu = 100)")
print("Alternative Hypothesis (H1): The mean impact force for new helmets is not equal to 100 Gs. (mu != 100)\n")

t_statistic_helmet, p_value_helmet = stats.ttest_1samp(a=new_helmet_impact_forces, popmean=known_avg_impact_force, alternative='two-sided')

print(f"Sample Mean Impact Force: {new_helmet_impact_forces.mean():.2f}")
print(f"Known Average Impact Force: {known_avg_impact_force}")
print(f"T-statistic: {t_statistic_helmet:.3f}")
print(f"P-value (two-sided): {p_value_helmet:.3f}")

print()
if p_value_helmet < alpha:
    print(f"Since the p-value ({p_value_helmet:.3f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is significant evidence that the new helmet design's average impact force is significantly DIFFERENT from 100 Gs.")
else:
    print()
    print(f"Since the p-value ({p_value_helmet:.3f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is not enough significant evidence to conclude that the new helmet design's average impact force is significantly DIFFERENT from 100 Gs.")

### 3.2. Two-Sample t-Test (`scipy.stats.ttest_ind`)

The two-sample (independent) t-test compares the means of two independent samples to determine if they are significantly different from each other. It's commonly used in A/B testing.


#### Task 10: Perform a 2-sample t-test.

**Scenario 1:** A team is considering two new offensive playbooks (Playbook A and Playbook B). They want to know if there's a significant difference in the average passing yards generated by QBs using Playbook A versus Playbook B.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Use `playbook_a_passing_yards` and `playbook_b_passing_yards` to perform the test. Set alpha = 0.05. Assume unequal variances (`equal_var=False`). Use the `alternative` parameter to get the correct p-value directly.

##### Dataset: Playbook Passing Yards

These two datasets, 'Playbook A Passing Yards' and 'Playbook B Passing Yards', represent the total passing yards accumulated by quarterbacks over 50 games when using two different offensive playbooks. The team wants to compare the effectiveness of these playbooks in generating passing offense.

In [ ]:
np.random.seed(43)
# Playbook A (Control Group)
playbook_a_passing_yards = pd.Series(np.random.normal(loc=250, scale=40, size=50)) # 50 games
# Playbook B (Treatment Group) - slightly higher mean
playbook_b_passing_yards = pd.Series(np.random.normal(loc=265, scale=45, size=50)) # 50 games


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that the average passing yards for Playbook A is equal to the average passing yards for Playbook B. This implies a two-sided test.

Use `stats.ttest_ind(a=sample1_data, b=sample2_data, equal_var=False, alternative='two-sided')`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): The average passing yards for Playbook A is equal to the average passing yards for Playbook B. (mu_A = mu_B)")
print("Alternative Hypothesis (H1): The average passing yards for Playbook A is not equal to the average passing yards for Playbook B. (mu_A != mu_B)\n")

t_statistic_2samp, p_value_2samp = stats.ttest_ind(
    a=playbook_a_passing_yards,
    b=playbook_b_passing_yards,
    equal_var=False, # Assuming unequal variances (Welch's t-test) which is often safer
    alternative='two-sided'
)

print(f"Mean Passing Yards (Playbook A): {playbook_a_passing_yards.mean():.2f}")
print(f"Mean Passing Yards (Playbook B): {playbook_b_passing_yards.mean():.2f}")
print(f"T-statistic (2-sample): {t_statistic_2samp:.3f}")
print(f"P-value (2-sample, two-sided): {p_value_2samp:.3f}")

print()
if p_value_2samp < alpha:
    print(f"Since the p-value ({p_value_2samp:.3f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is a significant difference in average passing yards between Playbook A and Playbook B.")
else:
    print()
    print(f"Since the p-value ({p_value_2samp:.3f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is no significant difference in average passing yards between Playbook A and Playbook B.")

#### Task 11: Perform a 2-sample t-test.

**Scenario 2:** A team wants to compare the average number of sacks allowed per game by their offensive line (OL) using two different blocking schemes: 'Zone Blocking' vs. 'Power Blocking'. They hypothesize that 'Zone Blocking' might lead to *fewer* sacks.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Generate synthetic data for sacks allowed under each scheme and perform the test. Set alpha = 0.05. Assume equal variances (`equal_var=True`) for this scenario. Use the `alternative` parameter to get the correct p-value directly.

##### Dataset: Blocking Scheme Sacks Allowed

These datasets represent the number of sacks allowed per game by an offensive line, under two different blocking schemes: 'Zone Blocking' and 'Power Blocking'. The team wants to determine if 'Zone Blocking' is more effective at reducing sacks, which would indicate an improvement in offensive line play.

In [ ]:
np.random.seed(48)
zone_blocking_sacks = pd.Series(np.random.normal(loc=2.0, scale=0.8, size=40))
power_blocking_sacks = pd.Series(np.random.normal(loc=2.5, scale=1.0, size=40))


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that the average sacks allowed for Zone Blocking is greater than or equal to Power Blocking. This implies a one-sided (left-tailed) test.

Use `stats.ttest_ind(a=sample1_data, b=sample2_data, equal_var=True, alternative='less')`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): The average sacks allowed for Zone Blocking is equal to or greater than Power Blocking. (mu_Zone >= mu_Power)")
print("Alternative Hypothesis (H1): The average sacks allowed for Zone Blocking is less than Power Blocking. (mu_Zone < mu_Power)\n")

t_statistic_sacks, p_value_sacks_one_sided = stats.ttest_ind(
    a=zone_blocking_sacks,
    b=power_blocking_sacks,
    equal_var=True, # Assuming equal variances
    alternative='less'
)

print(f"Mean Sacks Allowed (Zone Blocking): {zone_blocking_sacks.mean():.2f}")
print(f"Mean Sacks Allowed (Power Blocking): {power_blocking_sacks.mean():.2f}")
print(f"T-statistic: {t_statistic_sacks:.3f}")
print(f"One-sided (Left-tailed) P-value: {p_value_sacks_one_sided:.3f}")

print()
if p_value_sacks_one_sided < alpha:
    print(f"Since the one-sided p-value ({p_value_sacks_one_sided:.3f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is significant evidence that Zone Blocking leads to significantly FEWER sacks allowed compared to Power Blocking.")
else:
    print()
    print(f"Since the one-sided p-value ({p_value_sacks_one_sided:.3f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is not enough significant evidence to conclude that Zone Blocking leads to significantly FEWER sacks allowed.")

### 3.3. Chi-Squared Test of Independence (`scipy.stats.chi2_contingency`)

The Chi-squared test of independence is used to determine if there is a significant association between two categorical variables. In A/B testing, this could be used to see if a change (e.g., a new stadium type) affects the distribution of outcomes (e.g., win/loss).


#### Task 12: Perform a Chi-squared test of independence.

**Scenario 1:** A new NFL stadium with a retractable dome is being evaluated. We want to know if the type of stadium (Classic Open-Air vs. Modern Dome) is independent of the game outcome (Win/Loss) for a particular team.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Use `stadium_outcome_df` to create a contingency table, then apply `stats.chi2_contingency`. Set alpha = 0.05.

##### Dataset: Stadium Type and Game Outcome

This dataset contains information about the stadium type (Classic Open-Air or Modern Dome) and the game outcome (Win or Loss) for a series of games played by a particular NFL team. We want to investigate if there's a relationship between the type of stadium and the team's ability to win or lose games, which could inform decisions about future stadium designs or game strategies.

In [ ]:
np.random.seed(44)
stadium_types = ['Classic Open-Air', 'Modern Dome']
game_outcomes = ['Win', 'Loss']

# Simulate game outcomes for each stadium type
data_chi2 = []
num_games_per_stadium = 100

# Classic Open-Air: Slightly more losses due to weather variability
for _ in range(num_games_per_stadium):
    outcome = np.random.choice(game_outcomes, p=[0.49, 0.51]) # Adjusted for less extreme difference
    data_chi2.append({'stadium_type': 'Classic Open-Air', 'outcome': outcome})

# Modern Dome: Slightly more wins due to controlled environment
for _ in range(num_games_per_stadium):
    outcome = np.random.choice(game_outcomes, p=[0.50, 0.50]) # Adjusted for less extreme difference
    data_chi2.append({'stadium_type': 'Modern Dome', 'outcome': outcome})

stadium_outcome_df = pd.DataFrame(data_chi2)


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that game outcome is independent of stadium type. This implies a two-sided test.

First, create a contingency table using `pd.crosstab(index=df['col1'], columns=df['col2'])`. Then pass this table to `stats.chi2_contingency()`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): Game outcome is independent of stadium type.")
print("Alternative Hypothesis (H1): Game outcome is dependent on stadium type (i.e., there is an association).\n")

# Create a contingency table
contingency_table = pd.crosstab(stadium_outcome_df['stadium_type'], stadium_outcome_df['outcome'])
print("Contingency Table:\n", contingency_table)

chi2, p_value_chi2, dof, expected = stats.chi2_contingency(contingency_table)

print()
print(f"Chi-squared Statistic: {chi2:.3f}")
print(f"P-value: {p_value_chi2:.3f}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies (under H0):\n", expected)

print()
if p_value_chi2 < alpha:
    print(f"Since the p-value ({p_value_chi2:.3f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is a significant association between stadium type and game outcome.")
else:
    print()
    print(f"Since the p-value ({p_value_chi2:.3f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is no significant association between stadium type and game outcome (they are independent).")

#### Task 13: Perform another Chi-squared test of independence.

**Scenario 2:** A marketing team wants to understand if there's a relationship between the type of merchandise a fan prefers (Jerseys or Hats) and how often they attend live games (Attends Regularly or Attends Rarely). This could help tailor marketing campaigns.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Generate synthetic data for fan preferences and attendance, create a contingency table, and perform the test. Set alpha = 0.05.

##### Dataset: Fan Merchandise Preference and Game Attendance

This dataset captures the preferred merchandise type (Jerseys or Hats) and game attendance habits (Attends Regularly or Attends Rarely) for 200 surveyed NFL fans. The marketing team is interested in understanding if there's a relationship between the type of merchandise a fan prefers and how often they attend live games, which could help tailor marketing campaigns.

In [ ]:
np.random.seed(49)
merchandise_types = ['Jerseys', 'Hats']
attendance_levels = ['Attends Regularly', 'Attends Rarely']

data_marketing = []
num_fans = 500 # Increased sample size

for _ in range(num_fans):
    merch = np.random.choice(merchandise_types)
    if merch == 'Jerseys':
        # Slightly more likely to attend regularly if they buy jerseys
        attend = np.random.choice(attendance_levels, p=[0.51, 0.49]) # Adjusted for less extreme difference
    else:
        # Slightly less likely to attend regularly if they buy hats
        attend = np.random.choice(attendance_levels, p=[0.49, 0.51]) # Adjusted for less extreme difference
    data_marketing.append({'merchandise': merch, 'attendance': attend})

fan_data_df = pd.DataFrame(data_marketing)


In [ ]:
#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

The Null Hypothesis (H0) is that preferred merchandise type is independent of game attendance. This implies a two-sided test.

Follow the same steps as Task 12: create a contingency table using `pd.crosstab` and then use `stats.chi2_contingency`.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): Preferred merchandise type is independent of game attendance.")
print("Alternative Hypothesis (H1): Preferred merchandise type is dependent on game attendance (i.e., there is an association).\n")

contingency_table_marketing = pd.crosstab(fan_data_df['merchandise'], fan_data_df['attendance'])
print("Marketing Contingency Table:\n", contingency_table_marketing)

chi2_marketing, p_value_marketing, dof_marketing, expected_marketing = stats.chi2_contingency(contingency_table_marketing)

print()
print(f"Chi-squared Statistic: {chi2_marketing:.3f}")
print(f"P-value: {p_value_marketing:.10f}") # Increased precision
print(f"Degrees of Freedom: {dof_marketing}")

print()
if p_value_marketing < alpha:
    print(f"Since the p-value ({p_value_marketing:.10f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is a significant association between preferred merchandise type and game attendance.")
else:
    print()
    print(f"Since the p-value ({p_value_marketing:.10f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is no significant association between preferred merchandise type and game attendance.")

### 3.4. Chi-Squared Test with Multiple Groups and Post-Hoc Analysis

When a Chi-squared test with more than two groups (either rows or columns in the contingency table) yields a significant result, it tells us that there is *some* association between the variables. However, it doesn't tell us *which specific groups* are different from each other. For this, we need to perform **post-hoc analysis**.

Post-hoc tests for Chi-squared often involve performing pairwise Chi-squared tests (or examining adjusted standardized residuals) and then applying a **multiple comparison correction** (e.g., Bonferroni correction) to the p-values. This correction helps control the family-wise error rate (the probability of making at least one Type I error across all comparisons).


#### Task 14: Perform a Chi-squared test of independence with multiple groups and then conduct post-hoc analysis.

**Scenario:** An NFL team is evaluating three different fan engagement strategies (Strategy A, Strategy B, Strategy C) to see if they influence fan sentiment (Positive, Neutral, Negative) towards the team. They want to know if there's a significant association between the engagement strategy and fan sentiment, and if so, which strategies differ in their sentiment outcomes.

First, state your Null (H0) and Alternative (H1) Hypotheses for this scenario.

Use `fan_sentiment_df` to create a contingency table. Perform the initial Chi-squared test. If significant, perform pairwise Chi-squared tests between all combinations of strategies, applying a Bonferroni correction to the p-values. Set alpha = 0.05 for the initial test.

##### Dataset: Fan Engagement Strategy and Sentiment

This dataset simulates the sentiment (Positive, Neutral, Negative) of fans after being exposed to one of three different fan engagement strategies (Strategy A, Strategy B, Strategy C). The team wants to understand if certain strategies lead to more positive or negative sentiment, which will inform future marketing and fan interaction efforts.

In [ ]:
np.random.seed(50)
strategies = ['Strategy A', 'Strategy B', 'Strategy C']
sentiments = ['Positive', 'Neutral', 'Negative']

data_sentiment = []
num_fans_per_strategy = 200 # Total 600 fans

for _ in range(num_fans_per_strategy):
    sentiment = np.random.choice(sentiments, p=[0.40, 0.35, 0.25])
    data_sentiment.append({'strategy': 'Strategy A', 'sentiment': sentiment})

# Strategy B: More neutral
for _ in range(num_fans_per_strategy):
    sentiment = np.random.choice(sentiments, p=[0.30, 0.45, 0.25])
    data_sentiment.append({'strategy': 'Strategy B', 'sentiment': sentiment})

# Strategy C: Slightly more negative
for _ in range(num_fans_per_strategy):
    sentiment = np.random.choice(sentiments, p=[0.25, 0.35, 0.40])
    data_sentiment.append({'strategy': 'Strategy C', 'sentiment': sentiment})

fan_sentiment_df = pd.DataFrame(data_sentiment)

#Place your code here

##### Hint
<details>
<summary>Click to reveal hint</summary>

**For the initial overall test:**
1.  State your Null (H0) and Alternative (H1) Hypotheses.
2.  Construct a contingency table from `fan_sentiment_df` using `pd.crosstab()`, with 'strategy' as index and 'sentiment' as columns.
3.  Apply `scipy.stats.chi2_contingency()` to this table.

**If the initial test is significant (p-value < alpha):**
4.  Perform pairwise Chi-squared tests between all unique combinations of the three strategies (e.g., A vs B, A vs C, B vs C). `itertools.combinations` can be helpful here.
5.  For each pair, create a temporary subset of the data containing only those two strategies and then create its corresponding contingency table.
6.  Collect the p-values from these pairwise tests.
7.  Apply a Bonferroni correction to these collected p-values using `statsmodels.stats.multitest.multipletests(p_values, alpha=your_alpha, method='bonferroni')`. This adjusts for the increased chance of Type I errors when performing multiple tests.
8.  Interpret the *corrected* p-values to identify which specific strategy pairs show a significant association.
</details>

##### Solution

In [ ]:
alpha = 0.05

print("Null Hypothesis (H0): Fan sentiment is independent of the engagement strategy.")
print("Alternative Hypothesis (H1): Fan sentiment is dependent on the engagement strategy (i.e., there is an association).\n")

# Create a contingency table for the overall test
contingency_table_sentiment = pd.crosstab(fan_sentiment_df['strategy'], fan_sentiment_df['sentiment'])
print("Overall Contingency Table:\n", contingency_table_sentiment)

chi2_overall, p_value_overall, dof_overall, expected_overall = stats.chi2_contingency(contingency_table_sentiment)

print(f"\nOverall Chi-squared Statistic: {chi2_overall:.3f}")
print(f"Overall P-value: {p_value_overall:.10f}")
print(f"Overall Degrees of Freedom: {dof_overall}")

print("\n--- Overall Test Conclusion ---")
if p_value_overall < alpha:
    print(f"Since the overall p-value ({p_value_overall:.10f}) is less than alpha ({alpha}), we reject the null hypothesis.")
    print("Conclusion: There is a significant association between engagement strategy and fan sentiment. Proceeding with post-hoc analysis.")

    # --- Post-Hoc Analysis with Bonferroni Correction ---
    print("\n--- Post-Hoc Analysis (Bonferroni Corrected) ---")
    pairwise_p_values = []
    strategy_pairs = []

    # Get all unique pairs of strategies
    from itertools import combinations
    for s1, s2 in combinations(strategies, 2):
        strategy_pairs.append(f'{s1} vs {s2}')
        subset_df = fan_sentiment_df[fan_sentiment_df['strategy'].isin([s1, s2])]
        subset_contingency_table = pd.crosstab(subset_df['strategy'], subset_df['sentiment'])

        # Perform chi-squared test on the subset
        chi2_pair, p_value_pair, _, _ = stats.chi2_contingency(subset_contingency_table)
        pairwise_p_values.append(p_value_pair)

    # Apply Bonferroni correction
    reject_bonferroni, p_values_corrected, _, _ = multipletests(pairwise_p_values, alpha=alpha, method='bonferroni')

    for i, (pair, p_uncorrected, p_corrected, reject) in enumerate(zip(strategy_pairs, pairwise_p_values, p_values_corrected, reject_bonferroni)):
        print(f"  {pair}:")
        print(f"    Uncorrected P-value: {p_uncorrected:.10f}")
        print(f"    Bonferroni Corrected P-value: {p_corrected:.10f}")
        if reject:
            print(f"    -> Significant difference (Reject H0) at alpha={alpha}")
        else:
            print(f"    -> No significant difference (Fail to Reject H0) at alpha={alpha}")

else:
    print(f"Since the overall p-value ({p_value_overall:.10f}) is greater than or equal to alpha ({alpha}), we fail to reject the null hypothesis.")
    print("Conclusion: There is no significant association between engagement strategy and fan sentiment. No post-hoc analysis is needed.")

## Congratulations!
You've completed the A/B Testing with NFL Data Analysis Exercises. You've explored key statistical distributions, central measures, and deviations, understood the power of the Central Limit Theorem, and applied various hypothesis tests to real-world (synthetic) scenarios. This knowledge is fundamental for making data-driven decisions in any field, including the exciting world of sports analytics!